Transport phenomena is the study of the transferring of momentum, energy and mass. Oftentimes these processes are coupled as in boiling water, where bubbles of air trapped in the water nucleate due to the elevated temperature reducing the solubility of gases, followed by the density differences between the bubble and water causing the bubble to rise. These processes are further mediated through convection currents caused by temperature gradients in the boiling water. 

In this example, we will look at a far simple example of energy transport, characterized as the change in temperature of a heat source placed in an infinitely large cooler surrounding region. We will assume periodic boundary conditions to overcome the limitation of finite sizes in compute and assume that the temperature in the source reduces at the same time.  (INSERT IMAGE HERE)

# Theory

For our problem, we will be basing our grid on rectilinear or cartesian coordinates. First, we will review the relevant transport equations. A more detailed explanation of each expression and derivation can be found in BSLK. We begin by writing a general energy transport equation through a control volume.

$$ change = accumulation + removal + generation + destruction $$

In our closed system, there is no generation, destruction and accumulation of energy, meaning that the change in energy over time can be written as

$ change = removal$

If we approximate that the energy lost from the addition of heat is lost through conduction, the removal of energy can be approximated using fourier's law, defined as 

$$ q = -k \nabla T $$

where $k (\frac{W}{mK})$ is the thermal conductivity of the matrix, $\nabla T (\frac{K}{m})$ is the temperature gradient between the heated area and its surroundings and $q (\frac{W}{m^2 K})$ is the heat flux through a surface. This is related to the energy change at each grid point over time as, 

$\frac{d E}{dt} = \nabla q$

Where $E (\frac{J}{m^3})$ is the energy density of a specific grid point and $t (s)$ is the time. This leads to the expression, 

$$ \frac{d E}{d t} = -k \nabla^2 T $$

The energy density of the heat source can be expressed from the heat capacity of the heat source, defined as $E = \rho c_p (T - T_{ref})$. If we make the further approximation that density $\rho (\frac{kg}{m^3})$ and specific heat capacity $\frac{J}{K kg}$ stay constant with temperature, we can rearrange the above expression to obtain the following PDE written as a function of temperature exclusively,

$$ \frac{dT}{dt} = -\frac{k}{\rho c_p} \nabla^2 T$$

Oftentimes, the factor $\frac{k}{\rho c_p}$ is condensed into the term $\alpha (\frac{m^2}{s})$ defining the thermal diffusivity of the system and will be used as an input parameter.

Now that we have laid out the basic theory being implemented, we shall turn to how we can convert math to something a computer can solve. There are derivatives of time and space that we will need to solve for. I will stick to the simplest ones here as the focus is on creating an illustration how the code looks different between implementations in different languages. However, the literature of how both are done is vast with (SOURCES PROVIDED).

## Finite difference

The simplest way to discretize a differential equation in space is using a finite difference technique. The core idea of this scheme is we convert the continous differential equations above into discrete sums and differences which we can solve. Commonly, this is done using a central difference scheme shown below for the first $f^1_x$ and second derivative $f^2_x$ for a function $f(x,y,t)$

\begin{align}
f^1_x &= \frac{f(x + h, y, t) - f(x - h, y, t)}{2h} \\
f^2_x &= \frac{f(x + h, y, t) - 2f(x,y,t) + f(x - h, y, t)}{h^2} 
\end{align}

Where $h$ is the spacing between grids. Central difference offers produced second order error compared to forward or backward difference which produce first order error.

## Forward Euler

Next, I look at how we can integrate over time. Multiple time integration schemes exist which are split into explicit and implicit schemes. Explicit schemes include techniques such as Forward Euler and the Runge-Kutta time integrators while implicit schemes include the Backwards Euler and Crank-Nicholson schemes. For now, we will stick to the Forward Euler technique. 

Forward Euler solves an initial value problem by iterating a value of a variable through time using the initial value and the change in the variable through time. Notationally, this looks like

\begin{align}
g(x, y, t_0) &= b \\
\frac{dg}{dt} &= f(x,y,t) \\
g(x, y , t + \Delta t) &= g(x, y, t) + \Delta t \frac{dg}{dt}
\end{align}

where $g(x, y, t)$ is a value of a function at time $t$ and location $(x,y)$, $\frac{dg}{dt}$ is the derivative of function $g$ with respect to time $t$ and is expressed as any function $f(x,y,t)$. The value of $g(x, y, t + \Delta t)$ is a sum of the value of $g(x, y, t)$ and the timestep over which the derivative $\frac{dg}{dt}$ is calculated.

The Forward Euler technique can be numerically unstable. Therefore care must be taken when selecting parameters for time and space integration and must fall within the CFL condition, defined as $C = \frac{u \Delta t}{\Delta x} < 1$.

## Python implementation

Using the established rules above, we move to generating a prototype in python. We assume the heat source places is a circle at the middle of the system with a radius that can be specified with the system containing periodic boundary conditions. 

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import HTML
import glob

from analysis_src.forward_euler import FD_timestep, periodic_boundary
from analysis_src.visualization import animate_colormap

In [2]:
def make_system(nx, ny, R, T_hot = 373, T_cold = 273):
    X, Y = np.meshgrid(*[np.arange(0, nx), np.arange(0, ny)])
    out = np.where((X - nx/2)**2 + (Y - ny/2)**2 <= R**2, T_hot, T_cold)
    return out

In [ ]:
## CHANGE THESE ##
nx = 200 
ny = 200
# keep square for now (nx == ny)
halo = 2
R = 50

timesteps = 500
dt = 0.001
dump_freq = 10

alpha = 0.01
T_hot = 373
T_cold = 273
## CHANGE THESE ##

dx = 1/nx
dy = 1/ny
C = alpha/(min(dx, dy)**2/dt)
convergence_crit = 1e-7
if C > 1:
    raise RuntimeWarning("CFL condition not fulfilled")

temperature_fields_time = np.empty((timesteps//dump_freq+1, nx, ny))
times = np.empty(timesteps//dump_freq+1)

grid_old = make_system(nx+2*halo, ny+2*halo, R, T_hot, T_cold)
grid_new = np.empty((nx+2*halo, ny+2*halo))

for time in range(0, timesteps+1):
    if time%dump_freq == 0:
        temperature_fields_time[time//dump_freq] = grid_old[halo:nx+halo, halo:ny+halo]
        times[time//dump_freq] = time
    
    grid_new = FD_timestep(grid_old, 1, 1, dt, alpha, halo)
    grid_new = periodic_boundary(grid_new, halo)

    diff = np.mean(grid_new - grid_old)
    if np.abs(diff) < convergence_crit:
        temperature_fields_time[time//dump_freq + 1] = grid_new[halo:nx+halo, halo:ny+halo]
        times[time//dump_freq+1] = time

        temperature_fields_time = temperature_fields_time[:time//dump_freq + 1, :, :]
        times = times[:time//dump_freq+1]
        print("Converged")
        break

    grid_old = grid_new.copy()

Converged


In [12]:
ani = animate_colormap(temperature_fields_time, times = times)
HTML(ani.to_jshtml())

# Single core implementation in C++

In [ ]:
from analysis_src.visualization import read_data

In [ ]:
path = "src"
data_paths = sorted(glob.glob(f"{path}/T*.txt"))

In [ ]:
data = read_data(data_paths[-1])
plt.imshow(data)
plt.colorbar()

# Multicore (CPU) implementation in C++